In [1]:
args = {}


import math

import gpytorch
from gpytorch.means import ZeroMean
from gpytorch.kernels import RBFKernel, PeriodicKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import VariationalELBO, PredictiveLogLikelihood

from horseshoe_gp.src.structural_sgp import VariationalGP, StructuralSparseGP
eval("from horseshoe_gp.src.structural_sgp import " + args.selector)
from horseshoe_gp.src.mean_field_hs import MeanFieldHorseshoe, VariatioalHorseshoe

from botorch.fit import fit_gpytorch_model
eval("from botorch.acquisition.analytic import " + args.acq_fun)

from utils import branin_rcos, six_hump_camel_back, hartman_6, goldstein_price, rosenbrock

# set up kernels
n_kernels = 5
means = [ZeroMean()] * n_kernels
kernels = [RBFKernel()] * n_kernels

n_inducing = 10
inducing_points = torch.linspace(0, 1, n_inducing)

# GP for each kernel
gps = []
for mean, kernel in zip(means, kernels):
    gp = VariationalGP(mean, kernel, inducing_points)
    gps.append(gp)

eval("selector = " + args.selector + "(dim=n_kernels, A=1., B=1.)")
eval("model = " + args.model + "(gps, selector)")

dim = 5
num_restarts = 3
num_raw_samples = 100

likelihood = GaussianLikelihood()
elbo = PredictiveLogLikelihood(likelihood, model, num_data=args.num_raw_samples)
optimizer = torch.optim.Adam(list(model.parameters()) + list(likelihood.parameters()), lr=0.01) 
    

TypeError: __init__() got an unexpected keyword argument 'descripition'

In [ ]:
for i in range(100):
    optimizer.zero_grad()
    selector.update_tau_lambda()
    output = model(train_x)
    loss = - elbo(output, train_y)
    loss.backward(retain_graph=True)
    optimizer.step()
    print("Iter: {} \t Loss: {:.2f}".format(i, loss.item()))

for tries in range(NUM_TRIALS):
    